In [ ]:
!pip install tensorflow opencv-python mtcnn keras-facenet scikit-learn joblib

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from keras_facenet import FaceNet
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import joblib
from google.colab.patches import cv2_imshow

In [ ]:
def load_and_preprocess_image(path, size=(160, 160)):
    img = cv2.imread(path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, size)
    return img_resized

def detect_faces(image):
    detector = MTCNN()
    return detector.detect_faces(image)

embedder = FaceNet()

def get_embedding(face_img_rgb):
    return embedder.embeddings([face_img_rgb])[0]

In [ ]:
from google.colab import files
import shutil

uploaded = files.upload()

os.makedirs("dataset", exist_ok=True)

for filename in uploaded.keys():
    person_name = ''.join([c for c in filename if not c.isdigit()]).replace('.jpg', '').replace('.png', '')
    person_folder = f'dataset/{person_name}'
    os.makedirs(person_folder, exist_ok=True)
    shutil.move(filename, os.path.join(person_folder, filename))

print("Imagens organizadas.")

In [ ]:
dataset_path = "dataset"
embeddings = []
labels = []

for person in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person)
    for img_name in os.listdir(person_path):
        img_path = os.path.join(person_path, img_name)
        img = load_and_preprocess_image(img_path)
        embedding = get_embedding(img)
        embeddings.append(embedding)
        labels.append(person)

X_train, X_test, y_train, y_test = train_test_split(embeddings, labels, test_size=0.2)
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
joblib.dump(knn, "knn_classifier.pkl")

print("Treinamento completo. Modelo salvo.")

In [ ]:
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = cv2.imread(img_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

detector = MTCNN()
faces = detector.detect_faces(img_rgb)

knn = joblib.load("knn_classifier.pkl")

for face in faces:
    x, y, w, h = face['box']
    face_crop = img_rgb[y:y+h, x:x+w]
    face_crop = cv2.resize(face_crop, (160, 160))
    embedding = get_embedding(face_crop)
    prediction = knn.predict([embedding])[0]

    cv2.rectangle(img, (x, y), (x+w, y+h), (0,255,0), 2)
    cv2.putText(img, prediction, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

cv2_imshow(img)